# 🚀 Apache Spark 3.5.3 - Configuración Profesional de Cluster

## 📋 **Guía Completa para Configuración y Análisis Distribuido**

### 🎯 **Objetivos del Notebook:**
1. **Configurar correctamente** Apache Spark 3.5.3 en modo cluster
2. **Integrar con Apache Hive** para análisis SQL distribuido
3. **Implementar mejores prácticas** de configuración y optimización
4. **Realizar análisis de datos** del mundo real con rendimiento óptimo

### 🏗️ **Arquitectura del Cluster:**

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   JupyterLab    │    │     Master      │    │   Worker 1&2    │
│  (Spark Driver) │◄──►│ (Cluster Mgr)   │◄──►│   (Executors)   │
│   Spark 3.5.3   │    │  Spark 3.5.3    │    │  Spark 3.5.3    │
│     2GB RAM     │    │    2GB RAM      │    │   2GB RAM c/u   │
└─────────────────┘    └─────────────────┘    └─────────────────┘
         │                       │                       │
         └───────────────────────┼───────────────────────┘
                                 │
                    ┌─────────────────┐
                    │   Hive Metastore │
                    │   (PostgreSQL)   │
                    │    Datos: HDFS   │
                    └─────────────────┘
```

### 📊 **Recursos y Configuración:**
- **Total RAM disponible:** 8GB (2GB × 4 contenedores)
- **Configuración recomendada:** Driver 1.2GB + Executors 800MB c/u
- **Overhead JVM:** ~20% reservado para gestión de memoria
- **Cores totales:** 8 (2 cores × 4 contenedores)

---


## 📚 **Fundamentos de Apache Spark 3.5.3**

### 🔍 **¿Qué es Apache Spark?**

Apache Spark es un **motor de análisis distribuido** diseñado para procesar grandes volúmenes de datos de manera eficiente. A diferencia de Hadoop MapReduce, Spark mantiene los datos en memoria entre operaciones, lo que resulta en un rendimiento hasta **100x más rápido**.

### 🏛️ **Arquitectura de Spark:**

#### **1. Driver Program (Conductor)**
- **Función:** Coordina la ejecución de la aplicación Spark
- **Responsabilidades:** 
  - Crear SparkContext
  - Planificar tareas
  - Distribuir código a executors
  - Recolectar resultados

#### **2. Cluster Manager (Gestor de Cluster)**
- **Función:** Gestiona recursos del cluster
- **Tipos:** Spark Standalone, YARN, Kubernetes, Mesos
- **En nuestro caso:** Spark Standalone Manager

#### **3. Executors (Ejecutores)**
- **Función:** Procesan datos y ejecutan tareas
- **Características:**
  - Corren en nodos worker
  - Mantienen datos en memoria/disco
  - Ejecutan tareas en paralelo

### 🚀 **Ventajas de Spark 3.5.3:**

1. **Adaptive Query Execution (AQE):** Optimización dinámica de consultas
2. **Dynamic Partition Pruning:** Filtrado inteligente de particiones
3. **Columnar Storage Support:** Mejor rendimiento con formatos como Parquet
4. **Improved Catalyst Optimizer:** Optimizador de consultas más inteligente
5. **Better Memory Management:** Gestión más eficiente de memoria

### 📈 **Casos de Uso Típicos:**
- **ETL (Extract, Transform, Load):** Procesamiento de datos batch
- **Análisis en tiempo real:** Streaming de datos
- **Machine Learning:** MLlib para algoritmos distribuidos
- **Análisis interactivo:** Consultas SQL sobre Big Data
- **Graph Processing:** Análisis de redes y grafos

---


In [ ]:
# 📦 Importación de Librerías y Configuración Inicial

import findspark
import os
import sys
from datetime import datetime
import warnings

# Configurar findspark para localizar Spark
findspark.init('/opt/spark')

# Importar librerías de PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf

# Suprimir warnings para una salida más limpia
warnings.filterwarnings('ignore')

# Información del entorno
print("🔧 CONFIGURACIÓN INICIAL COMPLETADA")
print("="*50)
print(f"📍 Spark Home: {os.environ.get('SPARK_HOME', '/opt/spark')}")
print(f"🐍 Python Version: {sys.version.split()[0]}")
print(f"🖥️ Hostname: {os.environ.get('HOSTNAME', 'jupyterlab')}")
print(f"⏰ Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n✅ Librerías importadas correctamente")
print("🚀 Listo para crear sesión Spark...")


In [ ]:
# 🚀 CREACIÓN DE SESIÓN SPARK - CONFIGURACIÓN OPTIMIZADA DE RECURSOS

print("🔄 Iniciando creación de sesión Spark...")
print("⚙️ Aplicando configuración optimizada de recursos")

# Detener cualquier sesión existente
try:
    spark.stop()
    print("🛑 Sesión anterior detenida correctamente")
except:
    print("🔍 No había sesión activa")

# CONFIGURACIÓN DE RECURSOS OPTIMIZADA
# Basada en 8GB RAM total disponible (2GB x 4 contenedores)
spark = SparkSession.builder \
    .appName("EducacionIT-Spark-Professional-v3.5.3") \
    .master("spark://master:7077") \
    .config("spark.driver.memory", "1200m") \
    .config("spark.driver.cores", "2") \
    .config("spark.driver.maxResultSize", "500m") \
    .config("spark.executor.memory", "800m") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.memoryFraction", "0.8") \
    .config("spark.executor.memoryStorageFraction", "0.5") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.shuffle.service.enabled", "false") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .config("spark.network.timeout", "300s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "200") \
    .enableHiveSupport() \
    .getOrCreate()

# Configurar nivel de logging
spark.sparkContext.setLogLevel("WARN")

# Información de la sesión creada
print("\n🎉 ¡SESIÓN SPARK CREADA EXITOSAMENTE!")
print("="*60)
print(f"🏷️  Aplicación: {spark.sparkContext.appName}")
print(f"🔗 Master URL: {spark.sparkContext.master}")
print(f"📊 Spark UI: {spark.sparkContext.uiWebUrl}")
print(f"🎯 App ID: {spark.sparkContext.applicationId}")
print(f"🗄️ Catálogo: {spark.conf.get('spark.sql.catalogImplementation')}")
print(f"🚀 Versión Spark: {spark.version}")

# Verificar configuración de recursos aplicada
print("\n📋 CONFIGURACIÓN DE RECURSOS:")
print(f"   💾 Driver Memory: {spark.conf.get('spark.driver.memory')}")
print(f"   🖥️ Driver Cores: {spark.conf.get('spark.driver.cores')}")
print(f"   📊 Max Result Size: {spark.conf.get('spark.driver.maxResultSize')}")
print(f"   💾 Executor Memory: {spark.conf.get('spark.executor.memory')}")
print(f"   🖥️ Executor Cores: {spark.conf.get('spark.executor.cores')}")
print(f"   📊 Executor Instances: {spark.conf.get('spark.executor.instances')}")
print(f"   🔄 Adaptive Query: {spark.conf.get('spark.sql.adaptive.enabled')}")
print(f"   📦 Shuffle Partitions: {spark.conf.get('spark.sql.shuffle.partitions')}")

print("\n🌟 URLs de Monitoreo:")
print("   🎛️ Spark Master UI: http://localhost:8080")
print("   📊 Spark Driver UI: http://localhost:4040")

print("\n✅ Spark configurado con recursos optimizados para el entorno")


## 📚 **Tutorial Paso a Paso: Operaciones Básicas con Spark**

### 🎯 **Objetivo del Tutorial:**
Aprender las operaciones fundamentales de Apache Spark a través de ejemplos prácticos, desde la creación de DataFrames hasta consultas SQL complejas.

### 📋 **Operaciones que Cubriremos:**
1. **Crear DataFrames** desde datos en memoria
2. **Definir esquemas** personalizados
3. **Operaciones de transformación** (select, filter, groupBy)
4. **Operaciones de acción** (show, collect, count)
5. **Consultas SQL** sobre DataFrames
6. **Joins** entre múltiples DataFrames
7. **Agregaciones** y funciones de ventana
8. **Escritura y lectura** de datos

### 🏗️ **Estructura del Dataset de Ejemplo:**
Crearemos un dataset empresarial simulado con las siguientes entidades:
- **👥 Empleados**: ID, nombre, departamento, salario, fecha de contratación
- **🏢 Departamentos**: ID, nombre, presupuesto, ubicación
- **📊 Ventas**: ID, empleado_id, producto, monto, fecha

---


In [ ]:
# 📊 PASO 1: CREAR DATAFRAMES CON ESQUEMAS PERSONALIZADOS

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
from pyspark.sql.functions import col, when, lit
from datetime import date
import time

print("📊 CREANDO DATAFRAMES DE EJEMPLO...")
print("="*45)

# 1. DEFINIR ESQUEMAS (SCHEMAS)
print("\n1️⃣ DEFINIENDO ESQUEMAS PERSONALIZADOS:")

# Esquema para empleados
empleados_schema = StructType([
    StructField("emp_id", IntegerType(), False),
    StructField("nombre", StringType(), False),
    StructField("departamento_id", IntegerType(), False),
    StructField("salario", DoubleType(), False),
    StructField("fecha_contratacion", DateType(), False)
])

# Esquema para departamentos
departamentos_schema = StructType([
    StructField("dept_id", IntegerType(), False),
    StructField("nombre_dept", StringType(), False),
    StructField("presupuesto", DoubleType(), False),
    StructField("ubicacion", StringType(), False)
])

# Esquema para ventas
ventas_schema = StructType([
    StructField("venta_id", IntegerType(), False),
    StructField("emp_id", IntegerType(), False),
    StructField("producto", StringType(), False),
    StructField("monto", DoubleType(), False),
    StructField("fecha_venta", DateType(), False)
])

print("✅ Esquemas definidos correctamente")

# 2. CREAR DATOS DE EJEMPLO
print("\n2️⃣ CREANDO DATOS DE EJEMPLO:")

# Datos de empleados
empleados_data = [
    (1, "Ana García", 1, 75000.0, date(2020, 1, 15)),
    (2, "Carlos López", 2, 85000.0, date(2019, 3, 22)),
    (3, "María Rodríguez", 1, 70000.0, date(2021, 6, 10)),
    (4, "Juan Pérez", 3, 90000.0, date(2018, 9, 5)),
    (5, "Laura Martín", 2, 82000.0, date(2020, 11, 18)),
    (6, "Pedro Sánchez", 3, 95000.0, date(2017, 4, 12)),
    (7, "Isabel Torres", 1, 73000.0, date(2021, 2, 28)),
    (8, "Miguel Ruiz", 2, 88000.0, date(2019, 8, 14))
]

# Datos de departamentos
departamentos_data = [
    (1, "Recursos Humanos", 500000.0, "Madrid"),
    (2, "Tecnología", 1200000.0, "Barcelona"),
    (3, "Ventas", 800000.0, "Valencia")
]

# Datos de ventas
ventas_data = [
    (1, 4, "Laptop Pro", 1299.99, date(2023, 1, 10)),
    (2, 6, "Smartphone", 899.99, date(2023, 1, 12)),
    (3, 4, "Tablet", 599.99, date(2023, 1, 15)),
    (4, 6, "Monitor 4K", 399.99, date(2023, 1, 18)),
    (5, 4, "Teclado Mecánico", 149.99, date(2023, 1, 20)),
    (6, 6, "Mouse Gaming", 79.99, date(2023, 1, 22)),
    (7, 4, "Webcam HD", 129.99, date(2023, 1, 25)),
    (8, 6, "Auriculares", 199.99, date(2023, 1, 28))
]

print(f"📋 Empleados: {len(empleados_data)} registros")
print(f"🏢 Departamentos: {len(departamentos_data)} registros") 
print(f"💰 Ventas: {len(ventas_data)} registros")

# 3. CREAR DATAFRAMES
print("\n3️⃣ CREANDO DATAFRAMES:")

empleados_df = spark.createDataFrame(empleados_data, empleados_schema)
departamentos_df = spark.createDataFrame(departamentos_data, departamentos_schema)
ventas_df = spark.createDataFrame(ventas_data, ventas_schema)

print("✅ DataFrames creados exitosamente")

# 4. VERIFICAR ESTRUCTURA DE LOS DATAFRAMES
print("\n4️⃣ VERIFICANDO ESTRUCTURA:")

print("\n👥 ESQUEMA - EMPLEADOS:")
empleados_df.printSchema()

print("\n🏢 ESQUEMA - DEPARTAMENTOS:")
departamentos_df.printSchema()

print("\n💰 ESQUEMA - VENTAS:")
ventas_df.printSchema()

# 5. MOSTRAR DATOS DE MUESTRA
print("\n5️⃣ DATOS DE MUESTRA:")

print("\n👥 EMPLEADOS (primeras 5 filas):")
empleados_df.show(5)

print("\n🏢 DEPARTAMENTOS:")
departamentos_df.show()

print("\n💰 VENTAS (primeras 5 filas):")
ventas_df.show(5)

print("\n🎉 ¡DataFrames creados y verificados exitosamente!")
print("📊 Listos para operaciones de transformación y análisis")
